### 3.0 preload raw byte string content from pickle
+ raw contains 45 meeting minutes from San Jose city concoil.

In [18]:
import pickle

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [19]:
raw      = load_obj('./council_minutes_dict.pkl')
articles = []
for (key,article) in raw.items():
    #article = byte_article.decode('ISO8859-1')
    articles.append(article)

### 3.1 search and separate content as per formal text pattern
+ identify and separate individual main topic and the content between.
+ identify sub topic within the content block.

+ uses regex as main usage.

In [20]:
# import regex and identify patterns for each content of interests.
import regex as re

# room for improvement. host defined pattern to backend/user defined pattern
# may subject to change between different pdf contents
patterns = {'maintitle':re.compile(r'\n{1}\s?(\d{0,2}[.]\s*.*?)\n'),
           'subtitle':re.compile(r'\n{1}\s?(\d?[.]\d?\s*.*)'),
           'dollar':re.compile('([$]\d+([,.]\d+)*)'),
           'item':re.compile(r'\s(\d+-\d+\s+.*)\n'),
           'section':re.compile(r'\s((Sections?)\s\d+[.]\d+[.]\d+)')}

+ identifing recoganizable key words for main title recognition

In [21]:
# 

mainTopicLS =['CEREMONIAL',
              'CEREMONIALS',
              'ITEMS',
              'CONSENT',
              'CALENDAR',
              'STRATEGIC',
              'SUPPORT',
              'COMMUNITY',
             'ECONOMIC',
             'DEVELOPMENT',
             'NEIGHBORHOOD',
             'SERVICES',
             'TRANSPORTATION',
             'AVIATION',
             'ENVIRONMENTAL',
             'UTILITY',
             'PUBLIC',
             'SAFETY',
             'REDEVELOPMENT',
             'SUCCESSOR',
             'AGENCY',
             'LAND',
             'USE']
# The line below removes duplicates
mainTopicLS = list(dict.fromkeys(mainTopicLS)) 

+ main method to turn a string into a dataframe with main and subtitle marked.

In [22]:
import pandas as pd
import math

def qcRework (BLOCKS,pickLS = [],articleID = 1):
    Blocks = [[b,0,0,articleID] for b in BLOCKS]
    Blocks_result = []
    i = 0
    j = 0

    for b in Blocks:
        word_ls = b[0].split(' ')
        word_ls = [w for w in word_ls if w!=' ']
        word_ls = [w for w in word_ls if w!='']
        word_ls = [w for w in word_ls if w!='\n']
        try:
            main = float(word_ls[0])
            sub = round((main-i)*100)
            # print(word_ls[0],sub)
            if sub%10 == 0 and j != 9:
                sub = round(sub/10)
            else:
                pass
            if math.floor(main)==i and sub == j+1:
                j = sub
                #print(word_ls[0],j)
        except:
            pass
        
        if len(word_ls)<2:
            b[1] = j
            b[2] = i
            Blocks_result.append(b)
            continue
        
        if word_ls[-1] == word_ls[-1].upper() and word_ls[1] in pickLS:
            i += 1
            j = 0
            b[2] = i
            Blocks_result.append(b)
            
        else:
            b[1] = j
            b[2] = i
            Blocks_result.append(b)
            
    df_blocks = pd.DataFrame(Blocks_result,columns = ['content','subID','mainID','artID'])
    return df_blocks

+ process all articles in the string list.

In [23]:
articles_in_blocks = []
i = 1

for a in articles:
    #print(i)
    BLOCKS = re.split(patterns['maintitle'],a)
    articles_in_blocks.append(qcRework(BLOCKS,mainTopicLS,i))
    i+=1

+ visual validation

In [33]:
df_test = articles_in_blocks[1]

df_test.iloc[46]['content']

' \n\n Recommendation: (a) Adopt a resolution amending the 2019-2020 Schedule of Fees and Charges  \n  (Resolution No. 72737, as amended) to reduce plan review, building permit, and impact  \n\n  fees to $0 for property owners who participate in the ADU Amnesty Program and who  \n\n  qualify for a Financial Hardship Exemption under the criteria set forth in San José  \n\n Municipal Code Sections 4.76.400 through 4.76.470. \n (b) Adopt a resolution amending the Schedule of Parkland Fees (Resolution No. 73587,  \n as amended) to reduce fees to $0 for property owners who participate in the ADU  \n\n Amnesty Program and who qualify for a Financial Hardship Exemption under the  \n\n criteria set forth in San José Municipal Code Sections 4.76.400 through 4.76.470. \n\n CEQA: Not a Project, File No. PP17-005, Adjustment to Fees, Rates and Fares without  \n changes to or expansion of services. (Planning, Building and Code Enforcement/Parks,  \n\n Recreation, and Neighborhood Services) \n \n\n 

### 3.2 pack up all the subtitle content into one string.

+ function to identify the all the subtitle content pack into one

In [25]:
def df_pack(df_messy):
    df_compact = pd.DataFrame(columns = df_messy.columns)
    lastrow = None

    for id,row in df_messy.iterrows():
        if lastrow is None:
            lastrow = row
            continue
        if (row['subID'] == lastrow['subID']) and row['mainID'] == lastrow['mainID'] and row['artID'] == lastrow['artID']:
            lastrow['content'] += row['content']
        else:
            df_compact = df_compact.append(lastrow,ignore_index = True)
            lastrow = row
                  
    df_compact = df_compact.append(lastrow,ignore_index = True)
    
    return df_compact

+ process all the articles block list into dataframes.

In [26]:
df_articles = []

for a in articles_in_blocks:
    df_articles.append(df_pack(a))

### 3.3 add logic columns for specific contents

+ findPattern: find all the patterns in a string and return a logic value
+ addColumn: add a logic column for a specific pattern recognition by using findPattern function

In [27]:
def findPattern(s,pattern):
    sub = re.findall(pattern,s)
    #rint(sub)
    return sub !=[]

def addColumn(df,pattern,colname = None):
    if colname is None:
        colname = 'col' + str(len(df.columns))
    
    df[colname] = df['content'].apply(findPattern,args=[pattern])
    return df      


+ add hasDollar, hasItem and hasSection for all the artical dataframe

In [28]:
for df in df_articles:
    df = addColumn(df,patterns['dollar'],'hasDollar')
    df = addColumn(df,patterns['item'],'hasItem')
    df = addColumn(df,patterns['section'],'hasSection')


+ visual validation.

### 3.4 join all dataset in one dataframe

In [29]:
df_minutes =pd.DataFrame(columns= df_articles[0].columns)
df_minutes = df_minutes.append(df_articles,ignore_index = True)
df_minutes

,content,subID,mainID,artID,hasDollar,hasItem,hasSection
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,False,True,False
1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,False,False,False
2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,False,False,False
3,2.1 Approval of City Council Minutes.None prov...,1,2,1,False,False,False
4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,False,False,True
...,...,...,...,...,...,...,...
1571,4.1 20-1638 Update on Fine Amounts for Illegal...,1,4,45,False,True,True
1572,7. NEIGHBORHOOD SERVICES 7.1 20-1640 Police ...,0,5,45,False,True,False
1573,8. COMMUNITY & ECONOMIC DEVELOPMENT 8.1 20-1...,0,6,45,True,True,False
1574,9. REDEVELOPMENT – SUCCESSOR AGENCY \n The S...,0,7,45,False,True,False


### 3.5 save and read
+ to and from feather file
+ need pyarrow installed.

In [30]:
import time

start = time.time()
df_minutes.to_feather('city_SanJose_Minutes.feather')
df_read = pd.read_feather('city_SanJose_Minutes.feather')
print(time.time()-start)

df_read

0.033911705017089844


,content,subID,mainID,artID,hasDollar,hasItem,hasSection
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,False,True,False
1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,False,False,False
2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,False,False,False
3,2.1 Approval of City Council Minutes.None prov...,1,2,1,False,False,False
4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,False,False,True
...,...,...,...,...,...,...,...
1571,4.1 20-1638 Update on Fine Amounts for Illegal...,1,4,45,False,True,True
1572,7. NEIGHBORHOOD SERVICES 7.1 20-1640 Police ...,0,5,45,False,True,False
1573,8. COMMUNITY & ECONOMIC DEVELOPMENT 8.1 20-1...,0,6,45,True,True,False
1574,9. REDEVELOPMENT – SUCCESSOR AGENCY \n The S...,0,7,45,False,True,False


In [31]:
df_read['artID'].value_counts()

39    50
26    50
25    50
6     48
23    47
10    47
44    46
18    45
9     45
21    45
20    45
5     44
2     44
3     43
34    41
32    41
45    41
31    40
22    40
33    40
12    40
42    40
30    38
38    37
8     37
13    37
4     36
43    36
40    36
17    35
29    35
35    34
15    33
7     33
14    32
41    32
28    28
36    27
16    26
1     25
27     2
11     2
37     1
19     1
24     1
Name: artID, dtype: int64

+ to and from csv file

In [32]:
start = time.time()
df_minutes.to_csv('city_SanJose_Minutes.csv')
df_read = pd.read_csv('city_SanJose_Minutes.csv')
print(time.time()-start)

df_read

0.11669087409973145


,Unnamed: 0,content,subID,mainID,artID,hasDollar,hasItem,hasSection
0,0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,False,True,False
1,1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,False,False,False
2,2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,False,False,False
3,3,2.1 Approval of City Council Minutes.None prov...,1,2,1,False,False,False
4,4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,False,False,True
...,...,...,...,...,...,...,...,...
1571,1571,4.1 20-1638 Update on Fine Amounts for Illegal...,1,4,45,False,True,True
1572,1572,7. NEIGHBORHOOD SERVICES 7.1 20-1640 Police ...,0,5,45,False,True,False
1573,1573,8. COMMUNITY & ECONOMIC DEVELOPMENT 8.1 20-1...,0,6,45,True,True,False
1574,1574,9. REDEVELOPMENT – SUCCESSOR AGENCY \n The S...,0,7,45,False,True,False
